## Parse pdf and generate question bank

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('100q.pdf')

data = loader.load()


In [2]:
from main_quiz import QAs
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=QAs)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', max_tokens=1500)
prompt = PromptTemplate(
    input_variables=["document"],
    template="Parse the document and capture all questions and answers. Question will come with a number in front, and there might be multiple answers but you should parse it as one string. Do not return anything else.\n {format_instructions}. Document:\n {document}.\n ",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
from langchain.callbacks import get_openai_callback
master = QAs(items=[])
for doc in data:
    _input = prompt.format_prompt(document=doc.page_content)
    with get_openai_callback() as cb:
        output = llm.predict(_input.to_string())
        current_qa = parser.parse(output)
    print(cb)
    print(output)
    master.items.extend(current_qa.items)


In [ ]:
assert len(master.items) == 100

import json
with open('data.json', 'w') as f:
    json.dump(master.dict(), f, indent=4)